# SpaceRocks

### Vectorized coordinate transformation and ephemeris calculation with robust unit handling.

To install, simply `pip install spacerocks`

In [ ]:
from spacerocks import Units, SpaceRock, Observe, Propagate
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy import units as u
%matplotlib inline

In [ ]:
{‘a’: 81.48404137238033,
 ‘e’: 0.7094507249841506,
 ‘i’: 37.132857449687194,
 ‘lan’: 167.6781294110762,
 ‘aop’: 214.62054824981132,
 ‘top’: 2457115.3524901397}

{‘a’: 2.261165508330344e-05,
 ‘adot’: 0.0626425293290428,
 ‘b’: -1.427200794683609e-05,
 ‘bdot’: -0.03955374150304765,
 ‘g’: 0.04386010793151914,
 ‘gdot’: 0.00034576080057385533}
t0 = 2456916.7656669617

In [21]:
from spacerocks import Units, SpaceRock
import pandas as pd

units = Units()
print('Default Units:')
units.current()
 
print('\n\nUser-Modified Units')
units.timeformat = 'jd'
units.current()

r = SpaceRock(a=81.48404137238033, 
              e=0.7094507249841506, 
              inc=37.132857449687194, 
              arg=214.62054824981132, 
              node=167.6781294110762, 
              t_peri=2457115.3524901397, 
              epoch=2456916.7656669617,
              name='2014 SK378',
              input_frame='barycentric',
              units=units)

Default Units:
Quantity             Unit           
---------------------------------------
distance             AU             
angle                deg            
timescale            utc            
timeformat           None           
speed                AU / d         
rotation_period      d              


User-Modified Units
Quantity             Unit           
---------------------------------------
distance             AU             
angle                deg            
timescale            utc            
timeformat           jd             
speed                AU / d         
rotation_period      d              


In [22]:
from skyfield.api import Topos, Loader

In [23]:
load = Loader('./Skyfield-Data', expire=False, verbose=False)
ts = load.timescale()
planets = load('de423.bsp')
earth = planets['earth']

In [24]:
def ecl_to_tel(x_ec, y_ec, z_ec, l0, b0):
    '''
    This is right
    '''
    sl = np.sin(l0)
    cl = np.cos(l0)
    sb = np.sin(b0)
    cb = np.cos(b0)

    xT = -sl*x_ec + cl*y_ec
    yT = -cl*sb*x_ec -sl*sb*y_ec + cb*z_ec
    zT = cl*cb*x_ec + sl*cb*y_ec + sb*z_ec

    return xT, yT, zT

In [28]:
earth = planets['earth']
earth += Topos('30.169 S', '70.804 W', elevation_m=2200)
ee = earth.at(t)
ee.ecliptic_velocity().au_per_d

array([[ 1.71428939e-03],
       [ 1.72202410e-02],
       [-8.73047092e-05]])

In [29]:
earth = planets['earth']
ee = earth.at(t)
ee.ecliptic_velocity().au_per_d

array([[ 1.79294821e-03],
       [ 1.70194469e-02],
       [-3.83407985e-07]])

In [30]:
o = Observe(r, obscode='W84')
l = o.ecliptic_longitude
b = o.ecliptic_latitude

t = ts.tt(jd=r.epoch.tt.jd)
earth = planets['earth']
earth += Topos('30.169 S', '70.804 W', elevation_m=2200)

ee = earth.at(t)

x0, y0, z0 = ee.ecliptic_xyz().au * u.au
vx0, vy0, vz0 = ee.ecliptic_velocity().au_per_d * u.au / u.d

xT, yT, zT = ecl_to_tel(r.x - x0, r.y - y0, r.z - z0, l, b)
vxT, vyT, vzT = ecl_to_tel(r.vx - vx0, r.vy - vy0, r.vz - vz0, l, b)

gamma = 1/zT
alpha = gamma * xT
beta = gamma * yT
dalpha = gamma * vxT
dbeta = gamma * vyT
dgamma = gamma * vzT

In [31]:
print(alpha, beta, gamma)
print(2.261165508330344e-05, -1.427200794683609e-05, 0.04386010793151914)

[2.25882917e-05] [-1.42356282e-05] [0.04385654] 1 / AU
2.261165508330344e-05 -1.427200794683609e-05 0.04386010793151914


In [32]:
print(dalpha, dbeta, dgamma)
print(0.0626425293290428, -0.03955374150304765, 0.00034576080057385533)

[-0.00052678] 1 / d [-0.00020055] 1 / d [-0.00025799] 1 / d
0.0626425293290428 -0.03955374150304765 0.00034576080057385533


In [5]:
r.calc_abg(obscode='W84')

In [6]:
r.dalpha

<Quantity [-0.00053625] 1 / d>

In [ ]:
1/0.044507228

You can also pass in just a single object or an array of objects. I'll try all of the TNOs reported to the MPC. I specified an observatory code, so a topocentric correction will be applied to the Earth's position.

### Analyzing all of the TNOs in the MPC

In [ ]:
TNOs = pd.read_json('https://minorplanetcenter.net/Extended_Files/distant_extended.json.gz')
#TNOs = TNOs[TNOs.Principal_desig.values.astype(str) == '2015 BP519']
rocks = SpaceRock(a=TNOs.a.values, 
                  e=TNOs.e.values, 
                  inc=TNOs.i.values, 
                  arg=TNOs.Peri.values, 
                  node=TNOs.Node.values, 
                  t_peri=TNOs.Tp.values, 
                  epoch=TNOs.Epoch.values,
                  H=TNOs.H.values, 
                  name=TNOs.Principal_desig.values,
                  delta_H = np.random.rand(len(TNOs)),
                  rotation_period = np.random.uniform(0.2, 0.5, len(TNOs)),
                  phi0 = np.random.rand(len(TNOs)) * 2 * np.pi,
                  input_frame='heliocentric',
                  units=units)

#p = Propagate(rocks, np.linspace(2378480.5, 2378490.5, 10), model=5)

#obs_decam = Observe(p, obscode='W84', NSIDE=[128])
#obs_magellan = Observe(p, obscode=304, NSIDE=[256])
#obs_geocenter = Observe(p)

In [ ]:
rocks.__dict__

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(obs_decam.epoch.jd, obs_decam.mag, color='black', zorder=1)
#ax.plot(obs_decam_norot.epoch.jd, obs_decam_norot.mag, color='red', zorder=2)
ax.set_ylabel('Apparent Magnitude', fontsize=14, labelpad=20)
ax.set_xlabel('JD', fontsize=14, labelpad=10)
#ax.set_xlim([2378480.5, 2378490.5])
#ax.set_ylim([27, 27.5])

In [ ]:
obs_decam.astropy_table()

In [ ]:
rarate = obs_geocenter.ra_rate.to(u.arcsec / u.hour)
decrate = obs_geocenter.dec_rate.to(u.arcsec / u.hour)

In [ ]:
np.sqrt(rarate**2 + decrate**2)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
_ = ax.hist(np.sqrt(rarate**2 + decrate**2).value, bins=100)
ax.set_xlabel(r'Rate of motion (arcsec per hour)', fontsize=14, labelpad=10)

In [ ]:
rocks = rocks[rocks.e > 0.001]

In [ ]:
rocks2 = SpaceRock(x=rocks.x.value, 
                   y=rocks.y.value, 
                   z=rocks.z.value, 
                   vx=rocks.vx.value, 
                   vy=rocks.vy.value, 
                   vz=rocks.vz.value,
                   epoch=rocks.epoch.jd,
                   H=rocks.H, 
                   name=rocks.name,
                   input_coordinates='cartesian',
                   input_frame='heliocentric',
                   input_angles='degrees')

In [ ]:
from astroquery.jplhorizons import Horizons

In [ ]:
Nyears = 10
startdate = Time('2017-01-01', scale='utc', format='iso')
testdates = Time(np.arange(startdate.jd, startdate.jd + Nyears*365.25, 1), scale='utc', format='jd' )
tno_id = 'Ceres'
TNO_Horizons = Horizons(id=tno_id)
elements = TNO_Horizons.elements()[0]
ephem_Horizons = Horizons(id=tno_id, location='W84',
                          epochs={'start':testdates[0].iso, 'stop':testdates[-1].iso, 'step':'1d'}).ephemerides()
TNO = SpaceRock(a=elements['a'],
                e=elements['e'],
                inc=elements['incl'],
                arg=elements['w'],
                node=elements['Omega'],
                t_peri=elements['Tp_jd'],
                epoch=elements['datetime_jd'],
                name=[tno_id],
                input_frame='heliocentric',
                units=units)
TNO_prop = Propagate(TNO, obsdates=testdates.jd, model=5)
TNO_predict = Observe(TNO_prop, obscode='W84')
pos_Horizons = SkyCoord(ephem_Horizons['RA'], ephem_Horizons['DEC'], frame='icrs', unit=(u.deg, u.deg))
pos_pred = SkyCoord(TNO_predict.ra.deg, TNO_predict.dec.deg, frame='icrs', unit=(u.deg, u.deg))
sep = pos_pred.separation(pos_Horizons)

In [ ]:
elem = Horizons(id=tno_id, epochs={'start':testdates[0].iso, 'stop':testdates[-1].iso, 'step':'1d'}).elements()

In [ ]:
TNO_prop.epoch[0]

In [ ]:
elem['a'][0]

In [ ]:
print(elem['datetime_jd'][0], TNO_prop.epoch[0])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, elem['a'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_prop.a, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('a', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, elem['incl'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_prop.inc.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('inc', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, elem['e'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_prop.e, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('inc', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, (elem['incl'] - TNO_prop.inc.deg), color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('inc residuals (arcsec)', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, (elem['a'] - TNO_prop.a), color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('a residual', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(18, 12))
ax[0].scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['RA_rate'], color='black', s=50)
ax[0].scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.ra_rate.to(u.arcsec/u.h), color='red', s=10)
ax[0].set_xlabel('JD', fontsize=14)
ax[0].set_ylabel('ra rate (arcsec/h)', fontsize=14)
ax[0].set_xlim([0, Nyears]);

ax[1].scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['DEC_rate'], color='black', s=50)
ax[1].scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.dec_rate.to(u.arcsec/u.h), color='red', s=10)
ax[1].set_xlabel('JD', fontsize=14)
ax[1].set_ylabel('dec rate (arcsec/h)', fontsize=14)
ax[1].set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(18, 12))
ax[0].scatter((testdates.jd - testdates.jd[0])/365.25, 
              ephem_Horizons['RA_rate'] - TNO_predict.ra_rate.to(u.arcsec/u.h), color='black', s=1)
ax[0].set_xlabel('JD', fontsize=14)
ax[0].set_ylabel('ra rate (arcsec/h)', fontsize=14)
ax[0].set_xlim([0, Nyears]);

ax[1].scatter((testdates.jd - testdates.jd[0])/365.25, 
              ephem_Horizons['DEC_rate'] - TNO_predict.dec_rate.to(u.arcsec/u.h), color='black', s=1)
ax[1].set_xlabel('JD', fontsize=14)
ax[1].set_ylabel('dec rate (arcsec/h)', fontsize=14)
ax[1].set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, sep.arcsec, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Diff Horizons - SpaceRocks', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['elong'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.elong.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Solar Elongation Angle', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           ephem_Horizons['elong'] - TNO_predict.elong.deg, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Solar Elongation Angle Residual (deg)', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['delta'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.delta, color='red', s=10)

ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Earth Distance', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))

ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           (TNO_predict.delta - ephem_Horizons['delta']).to(u.km), color='black', s=1)

ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('Earth Distance Residuals (km)', fontsize=14)
ax.set_xlim([0, Nyears]);
#ax.set_ylim([0, 200]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['RA'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.ra.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('ra', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           (ephem_Horizons['RA'] - TNO_predict.ra.deg) * 3600, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('ra residuals (arcsec)', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, ephem_Horizons['DEC'], color='black', s=50)
ax.scatter((testdates.jd - testdates.jd[0])/365.25, TNO_predict.dec.deg, color='red', s=10)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('dec', fontsize=14)
ax.set_xlim([0, Nyears]);

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))
ax.scatter((testdates.jd - testdates.jd[0])/365.25, 
           (ephem_Horizons['DEC'] - TNO_predict.dec.deg) * 3600, color='black', s=1)
ax.set_xlabel('JD', fontsize=14)
ax.set_ylabel('dec residuals (arcsec)', fontsize=14)
ax.set_xlim([0, Nyears]);

---